In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [87.8 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.laun

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jarb

--2022-07-30 20:16:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jarb
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-07-30 20:16:59 ERROR 404: Not Found.



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [5]:
from pyspark.sql.functions import to_date

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame or table to retrieve all the rows 
# where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
filtered_votes = df.filter("total_votes >= 20")
filtered_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|
|         US|   12318409|R2KVWAYBPWK1OV|B011KEPZG8|     919734058|Iblue Canvas Leat...|         Luggage|          5|    

In [8]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
rf_votes = filtered_votes.filter("helpful_votes / total_votes >= 0.50")
rf_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|
|         US|   12318409|R2KVWAYBPWK1OV|B011KEPZG8|     919734058|Iblue Canvas Leat...|         Luggage|          5|    

In [72]:
rf_votes.count()

6711

In [9]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)
reviews_paid = rf_votes.filter(rf_votes["vine"] == 'Y')
reviews_paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15225359| RPJ7C4HP3BHXN|B00JR4A83O|     603403193|Fossil Men's Esta...|         Luggage|          5|           25|         29|   Y|                N|Beautiful Leather...|This Fossil bag i...| 2015-08-01|
|         US|   53064080|R3SB08XK0M7993|B00SXKUIX8|     905946428|AmazonBasics 4-Pi...|         Luggage|          4|    

In [67]:
reviews_paid.count()

21

In [10]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
reviews_unpaid = rf_votes.filter(rf_votes["vine"] == 'N')
reviews_unpaid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|
|         US|   12318409|R2KVWAYBPWK1OV|B011KEPZG8|     919734058|Iblue Canvas Leat...|         Luggage|          5|    

In [68]:
reviews_unpaid.count()

6690

In [11]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
# Total Review
total_reviews = df.select(["review_id"]).count
total_reviews()

348657

In [12]:
# Paid Reviews
total_reviews_paid = reviews_paid.count()
reviews_paid.count()

21

In [13]:
# Unpaid Reviews
total_reviews_unpaid = reviews_unpaid.count()
reviews_unpaid.count()

6690

In [14]:
# The number of 5-star reviews paid
five_star_paid = reviews_paid.filter(reviews_paid["star_rating"] == "5")
five_star_paid.count()

10

In [15]:
# The number of 5-star reviews unpaid
five_star_unpaid = reviews_unpaid.filter(reviews_unpaid["star_rating"] == "5")
five_star_unpaid.count()

3448

In [46]:
all_vine = df.filter(df["Vine"] == "Y")
not_vine = df.filter(df["vine"] == "N")
fivestar = all_vine.filter(all_vine["star_rating"] == 5)
not_fivestar = not_vine.filter(not_vine["star_rating"] == 5)

In [50]:
# The percentage of 5-star reviews paid
fivestar_paid_perc = (fivestar.count()/all_vine.count()) * 100
print(fivestar_paid_perc)

52.21238938053098


In [52]:
# The percentage of 5-star reviews unpaid
fivestar_unpaid_perc = (not_fivestar.count()/not_vine.count()) * 100
print(fivestar_unpaid_perc)

62.12073720262145


In [71]:
all_vine.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52851692| RWXIAG7MG39J4|B00UUGICCW|     236790043|Sherpani Amelia B...|         Luggage|          5|            0|          0|   Y|                N|stylish as either...|Love this bag.  W...| 2015-08-31|
|         US|   52913856|R3CDJ91K76HWCY|B00M3TJDKE|     691863261|Tumi Tegra Lite M...|         Luggage|          5|    

In [70]:
all_vine.count()

904

In [64]:
not_vine.count()

347747

In [65]:
fivestar.count()

472

In [66]:
not_fivestar.count()

216023